*UE Learning from User-generated Data, CP MMS, JKU Linz 2025*
# Exercise 2: Collaborative Filtering + Implicit Feedback

In this exercise we create our first proper recommender: Item-based K-Nearest Neighbours (ItemKNN) on implicit feedback from the family of Collaborative Filtering algorithms. Please refer to the slides and the recording of the relevant UE session published on Moodle if you have any problems. Feel free to ask questions in the Discussions forum.

The assignment submission deadline is (see Moodle).

Make sure to rename the notebook according to the convention:\
LUD25_ex02_k<font color='red'>\<Matr. 12312127\></font>_<font color='red'>\<Kaufmann-Lukas></font>.ipynb

for example:

LUD25_ex02_k000007_Bond-James.ipynb

## Implementation
In this exercise you are required to write three functions each calling the previous. Every function will be graded separately. Insert your implementations into the templates provided. Please don't change the templates even if they are not pretty. Don't forget to test your implementation for correctness and efficiency (a single run of any function should not take more than 2 minutes).

Please only use libraries already imported in the notebook. **Feel free to experiment with the notebook, but clean it up before submitting.**
    
The asserts present are meant for the LFM-tiny dataset from the previous exercise.

## Item-Based Collaborative Filtering
The idea of Item-Based Collaborative Filtering is to estimate if a user **u** is going to like item **i** through checking how similar this item is to the items already consumed (and rated) by the user. One way to calculate the estimation is a sum of ratings given by **u** to the consumed items weighted with their respective similarities to the item **i**. Please note, that from all items consumed by the user we only consider top **n** items most similar to item **i**.

In case of implicit feedback (which we deal with in this exercise), all "ratings" are either 0 or 1. Therefore, we combine our score from the similarities themselves. In this exercise we take the **average** of similarities to top **n** (or less if not enough neighbors were found) most similar to **i** items. Also note that we don't have to account for biases and missing values.

In [4]:
import pandas as pd
import numpy as np
from typing import Callable

## Interaction Matrix

In this exercise we work with the same format of interaction matrices as in the Exercise 1 (default settings) and the same data (LFM-tiny). Find a way to use the matrix in this notebook (e.g. copy your implementation of inter_matr_binary here and create the matrix anew).

In [34]:
def inter_matr_implicit(users: pd.DataFrame,
                        items: pd.DataFrame,
                        interactions: pd.DataFrame,
                        dataset_name: str,
                        threshold=1) -> np.ndarray:
    '''
    users - pandas Dataframe, use it as loaded from the dataset;
    items - pandas Dataframe, use it as loaded from the dataset;
    interactions - pandas Dataframe, use it as loaded from the dataset;
    dataset_name - string out of ["lfm-ismir", "ml-1m"], name of the dataset, used in case there are differences in the column names of the data frames;
    threshold - int > 0, criteria of a valid interaction

    returns - 2D np.ndarray, rows - users, columns - items;
    '''

    if dataset_name == "lfm-tiny":
        user_col = "user_id"
        item_col = "item_id"
        interaction_col = "listening_events"
    elif dataset_name == "ml-1m":
        user_col = "user_id"
        item_col = "item_id"  
        interaction_col = "rating"
    else:
        raise ValueError("Dataset name must be 'lfm-tiny' or 'ml-1m'")
    
    # User- und Item-Mappings erstellen
    user_ids = users[user_col].unique()
    item_ids = items[item_col].unique()
    
    user_map = {uid: i for i, uid in enumerate(user_ids)}
    item_map = {iid: i for i, iid in enumerate(item_ids)}
    
    # Interaktionsmatrix initialisieren
    interaction_matrix = np.zeros((len(user_ids), len(item_ids)), dtype=np.int8)
    
    # Interaktionsdaten filtern (nur relevante Spalten)
    interactions_filtered = interactions[[user_col, item_col, interaction_col]]
    
    # Nur Interaktionen über dem Schwellenwert behalten
    interactions_filtered = interactions_filtered[interactions_filtered[interaction_col] >= threshold]
    
    # Eintragen der Interaktionen in die Matrix
    for _, row in interactions_filtered.iterrows():
        user_idx = user_map.get(row[user_col])
        item_idx = item_map.get(row[item_col])
        if user_idx is not None and item_idx is not None:
            interaction_matrix[user_idx, item_idx] = 1  # Binäre Interaktion
    
    return interaction_matrix


In [36]:
# load the data for both datasets, keep it as specified in the csv files
def read(dataset, file):
    return pd.read_csv(dataset + '/' + dataset + '.' + file, sep='\t')

users = read("lfm-tiny", 'user')
items = read("lfm-tiny", 'item')
interactions = read("lfm-tiny", 'inter')



_interaction_matrix_test = inter_matr_implicit(users, items, interactions, "lfm-tiny", 1)
_interaction_matrix_test

array([[1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=int8)

## <font color='red'>TASK 1/3</font>: Get item similarities (3 points)
This is a helper function to be later used for user-item score estimation.
The function should take three arguments: a similarity measure (Jaccard distance in our case, see below), a binary interaction_matrix-like numpy array **inter** and a plane binary vector corresponding to an item **target_vec**. You can expect that the length of the vector corresponds to the number of users in the matrix **inter** (first parameter).

*the vector can be just a slice of the interaction matrix, see asserts*

Expected output: array of similarities between every item in **inter** and the vector **target_vec**. The similarities need to be placed in the order the items appear in the matrix **inter**.

Example: **inter** is a 7 by 3 matrix, containing information about 3 items and 7 users (can be expressed through item vectors as [it1; it2; it3]). **target_vec** is a vector of length 7 (assuming it tells us about interactions between the item and the same 7 users). The expected output is an array of length 3:\
[*sim*(it1, target_vec), *sim*(it2, target_vec), *sim*(it3, target_vec)]

**Similarity:** use jaccard score as the similarity measure. Please implement it yourself, don't use any external libraries;\
If $a$ and $b$ are two items, let's define $U(a)$ as the set of users, interacted with the item $a$ (same for $b$). $|U(a)|$ corresponds to the number of users interacted with the item $a$ (cardinality of the set). Then Jaccard similarity score between the two items is defined as:\
\
$JaccardScore(a,b) = \frac{|U(a) \wedge U(b)|}{|U(a) \vee U(b)|}$

In words: Jaccard Score is the number of users interacted with both items divided by the number of users interacted with at least one of them.

Use the cell below to define the similarity as a helper function, please don't change the parameters, name or output format.

<b>Tip:</b> The item vectors are on the axis=1 in the matrix.<br>

In [38]:
def jaccard_score(a: np.ndarray, b: np.ndarray) -> float:
    """
    a, b - 1D np.ndarray, vectors of the same length corresponding to the two items;

    returns - float, jaccard similarity score for a and b;
    """
    set_a = {i for i, val in enumerate(a) if val == 1}
    set_b = {i for i, val in enumerate(b) if val == 1}
    
    intersection = len(set_a & set_b)  # Common users interacted with both items
    union = len(set_a | set_b)  # Users who interacted with at least one item

    return intersection / union if union != 0 else 0.0  # Avoid division by zero

In [40]:
def calculate_sim_scores(similarity_measure: Callable[[np.ndarray, np.ndarray], float],
                         inter: np.ndarray,
                         target_vec: np.ndarray) -> np.ndarray:
    """
    similarity_measure - Callable, function that measures similarity, it gets called using your jaccard_score function from above - as always do not directly call your function, but use the passed parameter;
    inter - np.ndarray, interaction matrix - calculate similarity between each item and the target item (see below);
    target_vec - np.ndarray, target item vector;
    
    returns - np.ndarray, similarities between every item from <inter> and <target_vec> in the respective order;
    """
    return [similarity_measure(item, target_vec) for item in inter]

Now please call the function for the <b>whole</b> _interaction_matrix_test and the vector of the item with the <b>id 0</b>.

In [43]:
# here we pass your jaccard_score function as "similarity_measure" parameter
_interaction_matrix_test = inter_matr_implicit(users, items, interactions, "lfm-tiny", 1)

target_vec = _interaction_matrix_test[:, 0] 

item_sims = np.array(calculate_sim_scores(jaccard_score, _interaction_matrix_test.T, target_vec))

print(item_sims)

[1.         0.02083333 0.01030928 0.03529412 0.01449275 0.05555556
 0.04347826 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.04545455 0.
 0.         0.04545455 0.11764706 0.05882353 0.05882353 0.02150538
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.01898734 0.         0.         0.04545455 0.
 0.         0.         0.         0.01265823 0.         0.
 0.         0.         0.0625     0.         0.         0.05
 0.05555556 0.05555556 0.08695652 0.05769231 0.0625     0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.07142857 0.         0.         0.
 0.0625     0.01315789 0.0625     0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.04       0.         0.         0.         0.         0.03174603
 0.         0.         0.01234

In [45]:
assert item_sims is not None, "item_sims should not be None."
assert type(item_sims) == np.ndarray, "types are not correct."
assert len(item_sims) == 412, "length is not correct."
assert item_sims[0] == 1, "Item at the index 0 should have sim of 1"

## <font color='red'>TASK 2/3</font>: Estimate user-item score (3 points)

Write a function that takes a full interaction matrix as an input, as well as user id, item id and *n* -- algorithm's hyperparameter, number of neighbors to be considered while calculating the score.

The expected output is a single number between 0 and 1 - the predicted score.

Refer to the slides and the recording. Follow the algorithm:
* take items consumed by the user
* calculate the similarities between them and the target item (**exclude the target user from consideration** when calculating the similarities!)
* return **average** of top **n** with the highest similarity scores

<b>Tip:</b> Copy the interaction matrix before using it.<br>

In [48]:
def get_user_item_score(sim_scores_calculator: Callable[[np.ndarray, np.ndarray], np.ndarray],
                        inter: np.ndarray,
                        target_user: int,
                        target_item: int,
                        n: int = 2) -> float:
    """
    Estimate the user-item score based on item similarities.
    
    Parameters:
    sim_scores_calculator - Function that calculates similarity scores.
    inter - np.ndarray, interaction matrix.
    target_user - int, target user id.
    target_item - int, target item id.
    n - int, number of closest neighbors to consider.
    
    Returns:
    float, mean similarity score of top n most similar items.
    """
    # Copy interaction matrix to avoid modification
    inter_copy = inter.copy()
    
    # Get items the user has interacted with
    user_interactions = np.where(inter_copy[target_user] > 0)[0]
    
    # If no interactions, return 0
    if len(user_interactions) == 0:
        return 0.0
    
    # Compute similarity scores between interacted items and the target item
    item_similarities = sim_scores_calculator(inter_copy[:, user_interactions], inter_copy[:, target_item])
    
    # Sort similarities in descending order
    top_n_similarities = np.sort(item_similarities)[-n:]
    
    # Compute mean of top n similarities
    item_similarities_mean = np.mean(top_n_similarities) if len(top_n_similarities) > 0 else 0.0
    
    return item_similarities_mean



Now, call your function for <b>user 0</b> and <b>item 0</b> and <b> n = 10</b>

In [55]:
# you need to pass a "sim_scores_calculator" function into the "get_user_item_score" function,
# but "calculate_sim_scores" also takes a similarity measure function as parameter.
# The similarity measure is not necessarily present inside the "get_user_item_score" function
# Ideally you want to provide the similarity measure together with the "calculate_sim_scores" function

# The following line of code is one possible way to "bind" parameters to a function: you can now use the "sim_score_calc" function as parameter,
# which will always use your "jaccard_score" function as first parameter for "calculate_sim_scores" and passes through the other parameters
# This procedure is a way to keep your functions generic, you can now simply change your similarity measure via the
# function calls without needing to change the function bodies themselves
#
# Another advantage for you is that when we test your solution, we are going to pass our own implementations into your functions
# That means if you made a mistake in Task 1, you will still be able to get full points for consequent tasks if you did everything else correctly
# So make sure that your functions work independently of your other implemented functions by using the code we provide in this cell

def sim_score_calc(inter, target_vec):
    return calculate_sim_scores(jaccard_score, inter, target_vec)

# Generating the interaction matrix
_interaction_matrix_test = inter_matr_implicit(users, items, interactions, "lfm-tiny", 1)

# Get the interaction vector for the target item
target_vec = _interaction_matrix_test[:, target_item_id]  

# Calculate item similarity
item_sim = get_user_item_score(sim_score_calc, _interaction_matrix_test, target_user_id, target_item_id, n_neighbors)

print(item_sim)

0.011627906976744186


In [56]:
assert item_sim is not None, "item sim should have a value."
assert item_sim <= 1 and item_sim >= 0, "value of item sim is not valid."

## <font color='red'>TASK 3/3</font>: Recommender (4 points)

Implement the recommender function for the scoring algorithm you implemented above.
The function takes user_item_scorer (wrapper function already defined below), a full interaction matrix, user id, top_k, hyperparameter **n** as an input. It returns two arrays:

* top_k recommendations for the given user obtained considering **n** neighbors for score prediction
* the corresponding user-item similarity scores

Make sure you recommend items the user **hasn't seen** before! Try optimizing your implementation so that the runs take seconds rather than minutes.

Please don't change the "interface" of the function.

In [86]:
def recTopK(user_item_scorer: Callable[[Callable, np.ndarray, int, int], float],
            inter_matr: np.ndarray,
            user: int,
            top_k: int,
            n: int) -> (np.ndarray, np.ndarray):
    '''
    user_item_scorer - Callable, wrapper function that calculates user-item score, using get_user_item_score function
                                 from above, already defined in the next cell;
    inter_matr - np.ndarray, interaction matrix;
    user - int,  user_id;
    top_k - int, expected length of the resulting list;
    n - int, number of neighbors to consider;
    
    returns - 1D np.ndarray, of recommendations (sorted in the order of descending scores) & 1D np.ndarray, of corresponding scores;
    '''
    print("debug1")
    top_rec = None
    scores = None

    num_items = inter_matr.shape[1]  # Total number of items

    # Get items the user has already interacted with
    seen_items = np.where(inter_matr[user] > 0)[0]

    # Create a mask to exclude seen items
    all_items = np.arange(num_items)
    unseen_items = np.setdiff1d(all_items, seen_items)

    print("debug2")
    
    # Compute scores for unseen items
    scores = np.array([user_item_scorer(inter_matr, user, item, n) for item in unseen_items])

    # Get indices of the top_k highest scores
    top_k_indices = np.argsort(scores)[-top_k:][::-1]  # Sorting in descending order

    # Retrieve the top_k recommendations and their corresponding scores
    top_rec = unseen_items[top_k_indices]
    top_scores = scores[top_k_indices]
    
    return np.array(top_rec), np.array(top_scores)


Now, lets use these scoring functions and get the <b>top 10</b> recommendations for <b>user 0</b> with <b>n = 15</b>.

In [89]:
# see Task 2 for an explanation of the following function definition

top_k = 10
n = 15
user_id = 0  # Given user ID

def user_item_scorer(inter, target_user, target_item, n):
    return get_user_item_score(sim_score_calc, inter, target_user, target_item, n)

# Generate recommendations
rec_item_cf, scores_item_cf = recTopK(user_item_scorer, _interaction_matrix_test, target_user_id, top_k, n)


debug1
debug2


In [90]:
print("Recommendations with Item CF: ", rec_item_cf)
print("With Scores: ", scores_item_cf)
print("-" * 75)

Recommendations with Item CF:  [22 21 50 49 20 18 25 19 16 37]
With Scores:  [0.16666667 0.16666667 0.14828283 0.14828283 0.14285714 0.11111111
 0.1        0.09090909 0.09090909 0.07939394]
---------------------------------------------------------------------------


In [97]:
assert rec_item_cf is not None, "Recommendations should not be None."
assert type(rec_item_cf) == np.ndarray, "Types should be np.ndarray."
assert len(rec_item_cf) == 10, "10 recommendations should be returned."

In [99]:
assert scores_item_cf is not None, "Scores should not be None."
assert type(scores_item_cf) == np.ndarray, "Types should be np.ndarray."
assert len(scores_item_cf) == 10, "10 recommendations should be returned."

In [ ]:
# Leave this cell the way it is, please.